In [22]:
import bz2

bz_file = bz2.BZ2File("/home/a.melnikov/data.txt.bz2", 'r')
i = 0
c = 0
voc = dict()
prev_key = None
prev_approve = None
urlfrs = None
buf = set()
for line in bz_file:
    c += 1
    (phone, approve, urlfr, ymd, cnt) = line.decode("utf-8").strip().split('\t')
    key = '%s_%s' % (phone, approve)
    if key != prev_key:
        if prev_key is not None:
            for e in buf:
                if e not in voc:
                    voc[e] = (0, 0)
                if prev_approve == '1':
                    voc[e] = (voc[e][0], voc[e][1] + 1)
                else:
                    voc[e] = (voc[e][0] + 1, voc[e][1])
            #print('%s\t%d' % (prev_approve, wc))
        prev_key = key
        buf = set()
        prev_approve = approve
    buf.add(urlfr)
if prev_key is not None:
    for e in buf:
        if e not in voc:
            voc[e] = (0, 0)
        if prev_approve == '1':
            voc[e] = (voc[e][0], voc[e][1] + 1)
        else:
            voc[e] = (voc[e][0] + 1, voc[e][1])

len(voc)

499078719

In [24]:
c = 0
for e in voc:
    c += 1
    if c > 10:
        break
    print(e, voc[e])

e.mail.ru#14805776140000000049 (1, 0)
avito.ru#prodazha_telefonov_790953530 (2, 0)
artemushka.ru#pevica-valeriya-beremenna-foto (1, 0)
e.mail.ru#14622096630000000966 (1, 0)
vk.com#audios126775975 (1, 0)
vk.com#fotoandreyvalt (1, 0)
m.vk.com#photo287792060_429458151 (1, 0)
otvet.mail.ru#43352007 (3, 1)
rbc.ru#576296e39a7947699a017b1e (5, 10)
vk.com#id233002319 (2, 0)


In [27]:
a = 0.1
x = sorted([(k, voc[k], (voc[k][1] + a)/(voc[k][0] + a)) for k in voc if voc[k][0]+voc[k][1] > 100], key = lambda x: -x[2])

In [28]:
len(x)

1130800

In [29]:
x[:10]

[('finhow.ru#kak-popolnit-kartu-tinkoff-bez-komissii',
  (22, 82),
  3.7149321266968323),
 ('otzovik.com#review_3454893.html', (40, 142), 3.5436408977556106),
 ('direct-market.ru#kreditnaya-karta-tinkoff.html',
  (26, 84),
  3.222222222222222),
 ('frombanks.ru#minimalnyy-platyezh-po-karte-tinkoff',
  (26, 83),
  3.1839080459770113),
 ('otzovik.com#review_1247180.html', (43, 133), 3.088167053364269),
 ('otzovik.com#review_1063452.html', (87, 264), 3.032146957520092),
 ('lichnye-kabinety.ru#karta-tinkoff-lichnyj-kabinet',
  (60, 177),
  2.94675540765391),
 ('tinkoff-credity.ru#kak-zakryt-kartu-tinkoff', (26, 76), 2.915708812260536),
 ('tinkoffpro.ru#oplata-zhkx-s-pomoshhyu-tinkoff',
  (31, 90),
  2.8971061093247585),
 ('otzovik.com#review_1373450.html', (111, 321), 2.8901890189018906)]

In [30]:
x[-10:]

[('m.vk.com#write343274621', (127, 4), 0.03225806451612903),
 ('m.vk.com#id359922320', (101, 3), 0.03066271018793274),
 ('zaimizdes.ru#cherez-contact', (139, 4), 0.029475197699496764),
 ('m.vk.com#id379463536', (106, 3), 0.029217719132893498),
 ('zaimisrochno.ru#boostra', (108, 3), 0.028677150786308975),
 ('manimo.1kreditovanie.ru#zzz', (117, 3), 0.026473099914602907),
 ('rubanks.in#181.html', (131, 3), 0.02364607170099161),
 ('rubanks.in#77.html', (100, 2), 0.02097902097902098),
 ('m.vk.com#id312565547', (126, 2), 0.016653449643140365),
 ('m.vk.com#id337500679', (108, 1), 0.01017576318223867)]

In [32]:
with open('urlfr_coef.tsv', 'w') as f:
    for e in x:
        f.write('%s\t%d\t%d\t%f\n' % (e[0], e[1][0], e[1][1], e[2]))

In [33]:
!wc -l urlfr_coef.tsv

1130800 urlfr_coef.tsv


In [34]:
len(x)

1130800

In [35]:
!head urlfr_coef.tsv

finhow.ru#kak-popolnit-kartu-tinkoff-bez-komissii	22	82	3.714932
otzovik.com#review_3454893.html	40	142	3.543641
direct-market.ru#kreditnaya-karta-tinkoff.html	26	84	3.222222
frombanks.ru#minimalnyy-platyezh-po-karte-tinkoff	26	83	3.183908
otzovik.com#review_1247180.html	43	133	3.088167
otzovik.com#review_1063452.html	87	264	3.032147
lichnye-kabinety.ru#karta-tinkoff-lichnyj-kabinet	60	177	2.946755
tinkoff-credity.ru#kak-zakryt-kartu-tinkoff	26	76	2.915709
tinkoffpro.ru#oplata-zhkx-s-pomoshhyu-tinkoff	31	90	2.897106
otzovik.com#review_1373450.html	111	321	2.890189


In [2]:
x = {}
with open('urlfr_coef.tsv', 'r') as f:
    for line in f:
        (urlfr, c0, c1, coef) = line.strip().split('\t')
        c0 = int(c0)
        c1 = int(c1)
        coef = float(coef)
        x[urlfr] = ((c0, c1), coef)

In [3]:
len(x)

1130800

In [44]:
%%timeit

max_score = dict()
sum_score = dict()
cnt_score = dict()

bz_file = bz2.BZ2File("/home/a.melnikov/data.txt.bz2", 'r')
i = 0
print('start')
for line in bz_file:
    i += 1
    if i > 100:
        break
    
    (phone, approve, urlfr, ymd, cnt) = line.decode("utf-8").strip().split('\t')
    key = (phone, approve)
    if urlfr not in x:
        continue
    score = x[urlfr][2]
    if key not in max_score:
        if len(max_score) > 2:
            break
        max_score[key] = score
        sum_score[key] = score
        cnt_score[key] = 1 #fix0!
        continue
    max_score[key] = max(score, max_score[key])
    sum_score[key] += score
    cnt_score[key] += 1

len(max_score)

start
start
start
start
1 loop, best of 3: 7.72 s per loop


In [20]:
max_score = dict()
sum_score = dict()
cnt_score = dict()

i = 0
print('start')
with open('/home/a.melnikov/data100m.txt') as f:
    for line in f:
        i += 1
        (phone, approve, urlfr, ymd, cnt) = line.strip().split('\t')
        key = (phone, approve)
        if urlfr not in x:
            continue
        score = x[urlfr][1]
        if key not in max_score:
            max_score[key] = score
            sum_score[key] = score
            cnt_score[key] = 1 #fix0!
            continue
        max_score[key] = max(score, max_score[key])
        sum_score[key] += score
        cnt_score[key] += 1

len(max_score), i

start


(13549, 100000000)

In [40]:
cnt_score

{}

In [23]:
from sklearn.metrics import roc_auc_score

y = []
p = []
for k in max_score:
    y.append(int(k[1]))
    p.append(max_score[k])
roc_auc_score(y, p) #0.57273262661955249


0.58750752206915102

In [22]:
from sklearn.metrics import roc_auc_score

y = []
p = []
for k in sum_score:
    y.append(int(k[1]))
    p.append(sum_score[k]/(cnt_score[k])) #fix "-1"
roc_auc_score(y, p) #0.6555336595089245

0.64281022633676232

In [25]:
sum(y)/len(y)

0.2257731197874382

In [26]:
yp = sorted([(y[i], p[i]) for i in range(len(y))], key = lambda x: -x[1])

In [31]:
len(yp)

13549

In [29]:
yy = [e[0] for e in yp]
sum(yy[:6774])/6774

0.2679362267493357

In [34]:
sum(yy[:135])/135

0.6

In [35]:
0.6/0.2257731197874382

2.657535142203334

In [36]:
len(x)

1130800

In [80]:
import re

def transliterate(string):
    tr = {u'а': u'a',
           u'б': u'b',
           u'в': u'v',
           u'г': u'g',
           u'д': u'd',
           u'е': u'e',
           u'ё': u'e',
           u'ж': u'zh',
           u'з': u'z',
           u'и': u'i',
           u'й': u'y',
           u'к': u'k',
           u'л': u'l',
           u'м': u'm',
           u'н': u'n',
           u'о': u'o',
           u'п': u'p',
           u'р': u'r',
           u'с': u's',
           u'т': u't',
           u'у': u'u',
           u'ф': u'f',
           u'х': u'h',
           u'ц': u'ts',
           u'ч': u'ch',
           u'ш': u'sh',
           u'щ': u'sch',
           u'ъ': u'',
           u'ы': u'y',
           u'ь': u'',
           u'э': u'e',
           u'ю': u'yu',
           u'я': u'ya',}

    for k in tr:
        string = string.replace(k, tr[k])
        
    return string

def tri(s, d = dict()):
    s = transliterate(s.lower())
    prefix = 'D'
    for i in range(len(s)-2):
        if s[i] == '#':
            prefix = 'U'
            continue
        k = prefix + s[i:i+3]
        #if not k.isalnum():
        #if not k.isalpha():
        #    continue
        d[k] = d[k] + 1 if k in d else 1
    return d

In [81]:
t = dict()

for k in x:
    tri(k, t)
print(len(t))

76004


In [83]:
len([(k, t[k]) for k in t if t[k] > 100])

16357

In [84]:
sorted([(k, t[k]) for k in t if t[k] > 10], key = lambda x: -x[1])[-100:]

[('Ul51', 11),
 ('U-z5', 11),
 ('Di#i', 11),
 ('Dtbn', 11),
 ('Ub_4', 11),
 ('Ufuj', 11),
 ('Dh.j', 11),
 ('Dffp', 11),
 ('Ubw_', 11),
 ('Dbae', 11),
 ('D5.1', 11),
 ('Dayy', 11),
 ('Up_y', 11),
 ('Uhff', 11),
 ('Dbwo', 11),
 ('Ut4_', 11),
 ('Dlzi', 11),
 ('Urg8', 11),
 ('Ufuu', 11),
 ('Ud6k', 11),
 ('Dgcr', 11),
 ('Ua. ', 11),
 ('Dur1', 11),
 ('Ut. ', 11),
 ('Upbi', 11),
 ('Uffu', 11),
 ('D.a-', 11),
 ('Ucwi', 11),
 ('Uixp', 11),
 ('Dtdd', 11),
 ('U11 ', 11),
 ('Uy77', 11),
 ('U-rb', 11),
 ('Uvcl', 11),
 ('Uezc', 11),
 ('Uscy', 11),
 ('Uyzs', 11),
 ('U5.8', 11),
 ('D4hr', 11),
 ('D17-', 11),
 ('Ukv.', 11),
 ('Uz m', 11),
 ('Ue7-', 11),
 ('D.rp', 11),
 ('Dho4', 11),
 ('Dfko', 11),
 ('Dov6', 11),
 ('Dgyf', 11),
 ('Dv13', 11),
 ('Dj-v', 11),
 ('Up_4', 11),
 ('Dp2g', 11),
 ('Urll', 11),
 ('Ub_7', 11),
 ('Dbfk', 11),
 ('Uzso', 11),
 ('Ulji', 11),
 ('Ur74', 11),
 ('Uv d', 11),
 ('Uoaz', 11),
 ('D1a.', 11),
 ('Uya3', 11),
 ('Dhl-', 11),
 ('Ugrd', 11),
 ('Dusw', 11),
 ('Dm21', 11),
 ('Uxp_', 

In [85]:
tri_id = {}
for k in t:
    tri_id[k] = len(tri_id) + 1

In [86]:
tri_id['D701']

75816

In [93]:
prev_key = None
prev_approve = None
buf = dict()
with open('/home/a.melnikov/data1b.txt') as f:
    with open('/home/a.melnikov/data1b.libsvm', 'w') as fo:
        for line in f:
            (phone, approve, urlfr, ymd, cnt) = line.strip().split('\t')
            key = '%s_%s' % (phone, approve)
            if key != prev_key:
                if prev_key is not None:
                    fo.write('%s %s\n' % (prev_approve, ' '.join(['%d:%d' % (tri_id[k], buf[k]) for k in buf if k in tri_id])))
                prev_key = key
                buf = dict()
                prev_approve = approve
            tri(urlfr, buf)


In [91]:
!head -c200 ~/data1b.libsvm

0 18972:2 56982:47 49091:1 66466:62 19007:1 37952:5 47506:1 28393:13 37955:1 37957:20 18970:1 7335:2 28402:2 11:11 33141:1 56988:1 28405:4 66480:7 17:1 56992:3 20:1 21:21 36342:1 66486:1 58466:6 28417

In [92]:
!wc -l /home/a.melnikov/data1b.libsvm

1287 /home/a.melnikov/data1b.libsvm


In [1]:
sc.stop()